In [61]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [43]:
base_url = 'https://carigold.com/forum/forums/politics.108/'

In [65]:
def get_href(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    max_page = max([int(a_.split('page-')[-1]) for a_ in a if '/forum/forums/' in a_ and '/page-' in a_])
    a = [a_ for a_ in a if a_.startswith('/forum/threads/')]
    filtered = []
    for a_ in a:
        splitted = a_.split('/')
        if len(splitted) == 5 and splitted[-1] == '':
            filtered.append(a_)
    return filtered, max_page

filtered, max_page = get_href(base_url)

In [66]:
max_worker = 10

In [67]:
for i in tqdm(range(2, max_page + 1, max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls = [f'{base_url}page-{a}' for a in aranged]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, url): url for url in urls}

        for future in as_completed(futures):
            filtered.extend(future.result()[0])

100%|█████████████████████████████████████████| 116/116 [02:22<00:00,  1.23s/it]


In [70]:
topics = list(set(filtered))

In [72]:
topics[0]

'/forum/threads/shafie-bayangkan-warisan-bergerak-solo-di-sabah-semenanjung.749764/'

In [121]:
import time

def get_href_topic(url, topic = None):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    if topic:
        try:
            max_page = max([int(a_.split('page-')[-1]) for a_ in a if t in a_ and '/page-' in a_])
        except:
            max_page = -1
    else:
        max_page = -1
    div = soup.find_all('div', {'class': 'bbWrapper'})
    return [str(d) for d in div], max_page


t = topics[0]
url = f'https://carigold.com{t}'
filtered, max_page = get_href_topic(url, t)

In [122]:
import os
import json

!mkdir politics

mkdir: cannot create directory ‘politics’: File exists


In [123]:
sorted_topics = sorted(topics)

for no in tqdm(range(len(sorted_topics))):
    t = sorted_topics[no]
    
    filename = os.path.join('politics', f'{no}.json')
    if os.path.exists(filename):
        continue
    url = f'https://carigold.com{t}'
    current_issues, max_page = get_href_topic(url, t)
    
    for i in range(2, max_page + 1, max_worker):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page-{a}' for a in aranged]

        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href_topic, url): url for url in urls}

            for future in as_completed(futures):
                current_issues.extend(future.result()[0])
    
    with open(filename, 'w') as fopen:
        json.dump(current_issues, fopen)

 65%|█████████████████████▍           | 14954/23029 [3:50:11<5:02:51,  2.25s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 82%|██████████████████████████▉      | 18790/23029 [6:27:24<1:51:36,  1.58s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [124]:
!du -hs politics

1.9G	politics


In [126]:
# !zip -r politics.zip politics

In [129]:
!du -hs politics.zip

239M	politics.zip


In [130]:
from glob import glob

files = glob('politics/*.json')
len(files)

23029

In [131]:
files[0]

'politics/17887.json'

In [132]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [135]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    for d in data:
        texts.append(BeautifulSoup(d, "lxml").text)

100%|█████████████████████████████████████| 23029/23029 [05:49<00:00, 65.90it/s]


In [136]:
len(texts)

1912425

In [138]:
texts = list(set(texts))
len(texts)

1353425

In [139]:
with open('politics.json', 'w') as fopen:
    json.dump(texts, fopen)